In [15]:
# To reload external files automatically (not needed here)
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
from scipy import stats
import xesmf as xe # For regridding (https://xesmf.readthedocs.io/en/latest/)
import intake
from dask.distributed import Client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start dask client (for parallelization)

In [16]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:44319 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 33.33 GB


## Get multimodel data with Intake
https://intake-esm.readthedocs.io/en/latest/notebooks/tutorial.html

In [17]:
# Get the catalog (from Google Cloud)
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
print(type(col))
col

/home/lalandmi/miniconda3/envs/phd_v3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'intake_esm.core.esm_datastore'>


,unique
activity_id,15
institution_id,34
source_id,78
experiment_id,105
member_id,208
table_id,30
variable_id,391
grid_label,10
zstore,288908
dcpp_init_year,60


In [18]:
# Print all available experiments, models and tables
import pprint

uni_dict = col.unique(["source_id", "experiment_id", "table_id"])
pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 105,
                   'values': ['1pctCO2', '1pctCO2-bgc', '1pctCO2-rad',
                              'abrupt-0p5xCO2', 'abrupt-2xCO2', 'abrupt-4xCO2',
                              'abrupt-solm4p', 'abrupt-solp4p', 'amip',
                              'amip-4xCO2', 'amip-future4K', 'amip-hist',
                              'amip-lwoff', 'amip-m4K', 'amip-p4K',
                              'amip-p4K-lwoff', 'aqua-4xCO2', 'aqua-control',
                              'aqua-control-lwoff', 'aqua-p4K',
                              'aqua-p4K-lwoff', 'control-1950', 'dcppA-assim',
                              'dcppA-hindcast', 'dcppC-amv-ExTrop-neg',
                              'dcppC-amv-ExTrop-pos', 'dcppC-amv-Trop-neg',
                              'dcppC-amv-Trop-pos', 'dcppC-amv-neg',
                              'dcppC-amv-pos', 'dcppC-atl-control',
                              'dcppC-atl-pacemaker', 'dcppC-hindcast-noAgung',
                 

In [19]:
# Select the desired data
cat = col.search(
    experiment_id=["land-hist"],
    table_id="LImon",
    variable_id="snc",
#     source_id="IPSL-CM6A-LR",
#     member_id="r1i1p1f1"   
)

cat

/home/lalandmi/miniconda3/envs/phd_v3/lib/python3.8/site-packages/intake_esm/core.py:705: UserWarning: Query returned zero results.
  warn(message)


,unique
activity_id,0
institution_id,0
source_id,0
experiment_id,0
member_id,0
table_id,0
variable_id,0
grid_label,0
zstore,0
dcpp_init_year,0


In [13]:
cat.df.variable_id

0           prsn
1            tas
2         tasmax
3         tasmin
4          mrsol
5        cLitter
6       cProduct
7           cVeg
8     evspsblveg
9            gpp
10           lai
11          mrro
12         mrros
13          mrso
14         mrsos
15          tran
Name: variable_id, dtype: object

## Missing `snc` so finally I stay on CICLAD!

In [20]:
client.close()